# LeftJoin_Meteo_Estacion_1.0

En este script vamos a tratar de realizar un left join entre los datos meteorologicos y el listado de estaciones para en un mismo fichero tener la dupla estacion-coordenadas.

Para evitar generar duplicados, eliminaremos los mismos del fichero de estaciones (eliminaremos duplicados por nombre de estacion).

Una vez hecho esto, cruzamos los dos ficheros por las columnas "estacion"-"nombre" realizando un left join.

Generaremos un fichero llamado historico_estaciones_coordenadas.csv en el que albergaremos la informacion unificada.

In [1]:
#leemos los datasets de estaciones e historico de meteo:

import pandas as pd
import numpy as np

df_meteo = pd.read_csv('../DATASETS/ESTACIONES/historico_estaciones.csv',sep='|',error_bad_lines=False)
df_estaciones = pd.read_csv('../DATASETS/ESTACIONES/OMM_Estaciones.csv',sep='|',error_bad_lines=False)


In [2]:
df_meteo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 719228 entries, 0 to 719227
Data columns (total 12 columns):
Estacion            719228 non-null object
Fecha               719228 non-null object
T. med. (ºC)        719228 non-null object
T. max (ºC)         719228 non-null object
T. min (ºC)         719228 non-null object
Prec. (mm)          719228 non-null object
Presion/ Geopot.    719228 non-null object
Dir. vi.            719228 non-null object
Vel. vi. (Km/h)     719227 non-null object
Nub.                719227 non-null object
Prof. nieve (cm)    719227 non-null object
Insolac. (horas)    567417 non-null object
dtypes: object(12)
memory usage: 65.8+ MB


In [3]:
df_estaciones.head()

,IIiii,OACI,Nombre,Estado,Latitud,Longitud,Altitud,Notas
0,8427.0,NaN,ABLA,SPAIN,37-08-29N,002-46-46W,885,NaN
1,NaN,LETU,"ABLITAS, AERODROMO",SPAIN,42-00-40N,001-37-26W,324,NaN
2,8486.0,NaN,ADRA - CLUB NAUTICO,SPAIN,36-44-48N,003-01-14W,8,NaN
3,8432.0,NaN,AGUILAS,SPAIN,37-25-00N,001-35-12W,32,NaN
4,8019.0,NaN,AGUION,SPAIN,43-28N,006-18W,923,NaN


In [4]:
pd.concat(g for _, g in df_estaciones.groupby("Nombre") if len(g) > 1)

,IIiii,OACI,Nombre,Estado,Latitud,Longitud,Altitud,Notas
38,8087.0,NaN,BARDENAS REALES,SPAIN,42-12N,001-27W,295,NaN
39,NaN,LEBR,BARDENAS REALES,SPAIN,42-11-10N,001-29-02W,300,NaN
285,8235.0,NaN,TERUEL,SPAIN,40-21N,001-07W,900,NaN
286,NaN,LETL,TERUEL,SPAIN,40-24-43N,001-13-03W,1026,Desde 20131219


In [5]:
df_estaciones.sort_values("Nombre", inplace = True)
df_estaciones.drop_duplicates(subset ="Nombre", 
                     keep = False, inplace = True)

In [6]:
df_meteo.head()

,Estacion,Fecha,T. med. (ºC),T. max (ºC),T. min (ºC),Prec. (mm),Presion/ Geopot.,Dir. vi.,Vel. vi. (Km/h),Nub.,Prof. nieve (cm),Insolac. (horas)
0,LA CORUNA,31/01/2011,8.3,10.4,6.2,0.0,1024.1 Hpa,65º(NE),14,7/8,-,0.7
1,LUGO/ROZAS,31/01/2011,2.6,4.4,0.9,0.2,1024.3 Hpa,325º(NW),2,8/8,-,0.0
2,ASTURIAS/AVILES,31/01/2011,5.5,8.0,2.9,0.0,1025.0 Hpa,203º(SW),6,7/8,-,0.8
3,GIJON-MUSEL,31/01/2011,6.6,9.0,4.2,0.0,1024.1 Hpa,224º(SW),5,7/8,-,0.4
4,OVIEDO,31/01/2011,4.4,7.4,1.4,Ip,1024.5 Hpa,235º(SW),2,7/8,-,1.6


In [7]:
#left join para añadir al dataset de meteo las coordenadas de las estaciones

result = pd.merge(df_meteo, 
                  df_estaciones,
                  left_on='Estacion',
                  right_on='Nombre',
                  how='left')


In [8]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 719228 entries, 0 to 719227
Data columns (total 20 columns):
Estacion            719228 non-null object
Fecha               719228 non-null object
T. med. (ºC)        719228 non-null object
T. max (ºC)         719228 non-null object
T. min (ºC)         719228 non-null object
Prec. (mm)          719228 non-null object
Presion/ Geopot.    719228 non-null object
Dir. vi.            719228 non-null object
Vel. vi. (Km/h)     719227 non-null object
Nub.                719227 non-null object
Prof. nieve (cm)    719227 non-null object
Insolac. (horas)    567417 non-null object
IIiii               364786 non-null float64
OACI                17184 non-null object
Nombre              364786 non-null object
Estado              364786 non-null object
Latitud             364786 non-null object
Longitud            364786 non-null object
Altitud             364786 non-null float64
Notas               0 non-null object
dtypes: float64(2), object(18)
me

In [9]:
result.to_csv('../DATASETS/ESTACIONES/historico_estaciones_coordenadas.csv',sep='|',header=True,index=False)

## Analisis

Analizando el fichero que hemos obtenido se observa que ciertas estaciones meteorologicas no cruzan (por nombre) con el fichero obtenido de Luis Miguel Pascual "OMM_Estaciones.csv".

## Propuesta 

Se acude al codigo fuente de la pagina de donde extraemos los datos correspondientes a las estaciones meteorologicas.

De este código fuente se puede extraer el listado de estaciones y su ID asociado.

Montamos un fichero csv que contenga los "ID" y los "Nombre" de las estaciones (separado por pipes: '|').

Continua en el Script: LeftJoin_Meteo_Estacion_1.1